1. Neural Network from Scratch:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load MNIST dataset with the updated parser
mnist = fetch_openml('mnist_784', parser='auto')
X, y = mnist['data'], mnist['target'].astype(int)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)




class ImprovedNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights and biases with He initialization
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size)
        self.b2 = np.zeros((1, output_size))

    def relu(self, z):
        return np.maximum(0, z)

    def relu_derivative(self, z):
        return (z > 0).astype(float)

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / exp_z.sum(axis=1, keepdims=True)

    def cross_entropy_loss(self, y_pred, y_true):
        m = y_true.shape[0]
        return -np.sum(np.log(y_pred[np.arange(m), y_true])) / m

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.softmax(self.z2)
        return self.a2

    def backward(self, X, y_true, learning_rate=0.1):
        m = y_true.shape[0]

        # Compute gradients
        delta2 = self.a2
        delta2[np.arange(m), y_true] -= 1
        delta2 /= m

        dW2 = np.dot(self.a1.T, delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)

        delta1 = np.dot(delta2, self.W2.T) * self.relu_derivative(self.z1)
        dW1 = np.dot(X.T, delta1)
        db1 = np.sum(delta1, axis=0, keepdims=True)

        # Update weights and biases
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2

    def train(self, X_train, y_train, epochs=100, learning_rate=0.1):
        for epoch in range(epochs):
            y_pred = self.forward(X_train)
            loss = self.cross_entropy_loss(y_pred, y_train)
            self.backward(X_train, y_train, learning_rate)
            if epoch % 10 == 0:
                print(f'Epoch {epoch}: Loss = {loss}')

    def predict(self, X):
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)




input_size = 784  # 28x28 pixels
hidden_size = 64
output_size = 10  # Digits 0-9

# Initialize and train the neural network
nn = ImprovedNeuralNetwork(input_size=784, hidden_size=64, output_size=10)
nn.train(X_train, y_train, epochs=100, learning_rate=0.1)



# Evaluate the model
y_pred = nn.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')



Epoch 0: Loss = 3.015470190123657
Epoch 10: Loss = 0.9480035911413923
Epoch 20: Loss = 0.6486581371920429
Epoch 30: Loss = 0.5285277409028137
Epoch 40: Loss = 0.46217202915728584
Epoch 50: Loss = 0.4188946931900287
Epoch 60: Loss = 0.3877331350408779
Epoch 70: Loss = 0.3639294080318575
Epoch 80: Loss = 0.3448635350088612
Epoch 90: Loss = 0.32906870648465814
Test accuracy: 90.64%


2. Build with PyTorch

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class PyTorchNN(nn.Module):
    def __init__(self):
        super(PyTorchNN, self).__init__()
        self.fc1 = nn.Linear(784, 64)
        self.fc2 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

  # Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create DataLoader for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
model = PyTorchNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training loop
for epoch in range(100):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}: Loss = {loss.item()}')
# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_class = torch.argmax(y_pred, dim=1)
    accuracy = (y_pred_class == y_test_tensor).float().mean()
    print(f'Test accuracy: {accuracy.item() * 100:.2f}%')



Epoch 0: Loss = 1.5678085088729858
Epoch 10: Loss = 1.5043786764144897
Epoch 20: Loss = 1.4810359477996826
Epoch 30: Loss = 1.4700003862380981
Epoch 40: Loss = 1.4614520072937012
Epoch 50: Loss = 1.4771428108215332
Epoch 60: Loss = 1.4617832899093628
Epoch 70: Loss = 1.461221694946289
Epoch 80: Loss = 1.4613761901855469
Epoch 90: Loss = 1.4770283699035645
Test accuracy: 96.69%
